In [1]:
import requests
import pandas as pd
import os

# API-nøkkel og stasjon
CLIENT_ID = "2a430097-ac9b-44e4-995e-a02afb9795b2"
SOURCE = "SN68860"

# Dato-intervall
start_date = "2024-07-01"
end_date = "2024-12-31"
reference_time = f"{start_date}/{end_date}"

# Frost API-url
BASE_URL = "https://frost.met.no/observations/v0.jsonld"

# Elementer som skal hentes og deres etiketter
elements = {
    "air_temperature": "Temperatur (°C)",
    "mean(air_pressure_at_sea_level P1D)": "Lufttrykk (hPa)",
    "sum(precipitation_amount P1D)": "Nedbør (mm)"
}

# Brukerinput for å bestemme om man skal hente nye data
force_update = input("Vil du hente nye data fra Frost API? (ja/ne): ").strip().lower()

# Funksjon for å hente og lagre data
def fetch_data(element_id, label):
    filename = f"frost_{element_id.replace('(', '').replace(')', '').replace(' ', '_')}_jul-des_2024.csv"
    
    if os.path.exists(filename) and force_update != "ja":
        print("Bruker eksisterende fil:", filename)
        return

    print("Henter data for", label + "...")
    
    params = {
        "sources": SOURCE,
        "elements": element_id,
        "referencetime": reference_time
    }

    response = requests.get(BASE_URL, params=params, auth=(CLIENT_ID, ""))
    
    if response.status_code == 200:
        raw_data = response.json()["data"]
        rows = []
        for entry in raw_data:
            date = entry["referenceTime"].split("T")[0]
            value = entry["observations"][0]["value"]
            rows.append((date, value))
        
        df = pd.DataFrame(rows, columns=["Dato", label])
        df.to_csv(filename, index=False, encoding="utf-8")
        print("Lagret til:", filename)
    
    else:
        print(f"Feil ved henting av {element_id}: {response.status_code}")
        print(response.text)

# Hent og lagre data for hvert element
for element_id, label in elements.items():
    fetch_data(element_id, label)
    


Henter data for Temperatur (°C)...
Lagret til: frost_air_temperature_jul-des_2024.csv
Henter data for Lufttrykk (hPa)...
Lagret til: frost_meanair_pressure_at_sea_level_P1D_jul-des_2024.csv
Henter data for Nedbør (mm)...
Lagret til: frost_sumprecipitation_amount_P1D_jul-des_2024.csv
